In [1]:
%load_ext autoreload
%autoreload 2

from torch.utils.data import DataLoader

from lib.aperture_dataset import ApertureDataset

fname_train = 'training_data/training_data.h5'
fname_validate = 'training_data/validation_data.h5'

save_path = 'save_here'

model_params_dict = {
    'input_dim': 130, # TODO: move?
    'output_dim': 130,
    'layer_width': 260,
#     'num_hidden_layers': 5,
}

num_samples = 10 ** 5
dat_train = ApertureDataset(fname_train, num_samples, 4)
train_loader = DataLoader(dat_train, batch_size=1000, shuffle=True, num_workers=1)

num_samples = 10 ** 4
dat_train2 = ApertureDataset(fname_train, num_samples, 4)
train_loader2 = DataLoader(dat_train2, batch_size=1000, shuffle=False, num_workers=1)

num_samples = 10 ** 4
dat_validate = ApertureDataset(fname_validate, num_samples, 4)
validate_loader = DataLoader(dat_validate, batch_size=1000, shuffle=False, num_workers=1)

/Users/zhanwenchen/anaconda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from torch import optim

# from lib.fully_connected_net import FullyConnectedNet
from lib.fully_connected_net_5 import FullyConnectedNet
from lib.fit import fit

lr = 0.5
momentum = 0
nn_fc = FullyConnectedNet(**model_params_dict)
optimizer = optim.SGD(nn_fc.parameters(), lr, momentum)

fit(nn_fc, train_loader, train_loader2, validate_loader, optimizer, save_path, cuda=False)

E: 1 / Train: 3.423e-02 / Valid: 3.473e-02 / Diff Valid: -99.97% / Diff Valid-Train: 1.5% / Time: 3.58
E: 2 / Train: 1.910e-02 / Valid: 1.928e-02 / Diff Valid: -44.48% / Diff Valid-Train: 1.0% / Time: 3.63
E: 3 / Train: 1.337e-02 / Valid: 1.349e-02 / Diff Valid: -30.03% / Diff Valid-Train: 0.9% / Time: 3.65
E: 4 / Train: 1.042e-02 / Valid: 1.051e-02 / Diff Valid: -22.08% / Diff Valid-Train: 0.9% / Time: 3.59
E: 5 / Train: 8.515e-03 / Valid: 8.601e-03 / Diff Valid: -18.19% / Diff Valid-Train: 1.0% / Time: 3.69
E: 6 / Train: 7.540e-03 / Valid: 7.634e-03 / Diff Valid: -11.25% / Diff Valid-Train: 1.2% / Time: 3.77
E: 7 / Train: 6.594e-03 / Valid: 6.688e-03 / Diff Valid: -12.39% / Diff Valid-Train: 1.4% / Time: 3.70
E: 8 / Train: 5.993e-03 / Valid: 6.087e-03 / Diff Valid: -8.99% / Diff Valid-Train: 1.6% / Time: 3.80
E: 9 / Train: 5.515e-03 / Valid: 5.609e-03 / Diff Valid: -7.85% / Diff Valid-Train: 1.7% / Time: 3.81
E: 10 / Train: 5.085e-03 / Valid: 5.173e-03 / Diff Valid: -7.77% / Diff Val